In [1]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import default_rng
import scipy as sp
import scipy.special
import random
import torch

### Modifying training on the second teacher
Our new hypothesis is prioritising replay of rare and high reward inputs reduces forgetting. We can think of rare inputs (i.e. inputs which have strong presence in the input distribution of Task A but are uncommon in Task B) as similar to replaying unexplored regions. Mice have been shown to replay and imagine parts of a maze which they have seen but not explored e.g. they saw a left turn in T interchange but went right instead. 

We can modify the get_batch() function in the training_step() function.

Inline below, I copy the current get_batch function in our exponnorm and modify it to accept our values. 

Pipeline: Network_Runner
            -> train():
            ->train_on_step
            ->training_step:
                ->get_batch (called from a data_module as specified by the config)
                

We want the data distribution of the second input to shift each time, so that we cover a different section of the distribution. 

How do we want to systematically vary this filter?
Ideas:
- Very simple - Shift mean, keep K and std constant
- More interesting - vary K and the mean.

What if we could design the distribution parameters so that we have a different percentage of overlapped inputs each time?

e.g Given {p1, p2} in X2 (second distribution)  which are 2+ (or 3+) std (here std refers to the standard deviation of the exponorm!) away from the mean.
Then we can find the integral between these points and {-inf, inf} for both X1 and X2 and find the fraction of shared input probability?
Working backwards, we could vary the fraction and thus extract the value of K giving rise to this fraction -> we have our new distribution. 

Frankly, testing shows that varying K only and setting a constant std could be enough for our purposes...

Approach for this would be:
- Integrate Normal Distribution & exponnorm over range defined by std
- define fraction (one area/another) using constants and independant variable=std.
- solve. i.e. if we want frac = 0.2, what is our value for K?
- check how shifting the mean works with this, but it should be enough to just vary K. 